In [2]:
import requests
from json_to_psql import spark_to_sql
from pyspark.sql import SparkSession
from creds import read_cred_from_file

+------+----------------+------------+-------------+--------------------+-----------+----------+--------+----------+---------+--------------------+-----------+---------+-----------------+--------------------+
|APT_NO|  CREDIT_CARD_NO|   CUST_CITY| CUST_COUNTRY|          CUST_EMAIL| CUST_PHONE|CUST_STATE|CUST_ZIP|FIRST_NAME|LAST_NAME|        LAST_UPDATED|MIDDLE_NAME|      SSN|      STREET_NAME| FULL_STREET_ADDRESS|
+------+----------------+------------+-------------+--------------------+-----------+----------+--------+----------+---------+--------------------+-----------+---------+-----------------+--------------------+
|   656|4210653310061055|     Natchez|United States| AHooper@example.com|(123)-781-8|        MS|   39120|      Alec|   Hooper|2018-04-21T12:49:...|         wm|123456100|Main Street North|Main Street North...|
|   829|4210653310102868|Wethersfield|United States| EHolman@example.com|(123)-893-3|        CT|   06109|      Etta|   Holman|2018-04-21T12:49:...|    brendan|12345

In [3]:
# Send GET request to the URL
url = "https://raw.githubusercontent.com/platformps/LoanDataset/main/loan_data.json"
response = requests.get(url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    loan_data = response.json()  
  
    print("Loan data retrieved successfully!")
    print("Number of loan applications:", len(loan_data))
else:
    print("Failed to retrieve loan data. Status code:", response.status_code)

Loan data retrieved successfully!
Number of loan applications: 511


In [4]:
spark = SparkSession.builder \
    .appName("APItoMySQL") \
    .getOrCreate()

In [5]:
loan_df = spark.createDataFrame(loan_data)

db_name = 'creditcard_capstone'
table_name = 'CDW_SAPP_loan_application'
url = f'jdbc:mysql://localhost:3306/{db_name}'

username, password = read_cred_from_file()
properties = {
    'user': username,
    'password': password,
    'driver': 'com.mysql.jdbc.Driver'
}


In [6]:
spark.sql(f"CREATE DATABASE IF NOT EXISTS {db_name}")
spark.sql(f"USE {db_name}")

spark_to_sql(loan_df, table_name='cdw', url = url, properties= properties )
spark.stop()

---

# END